In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv('.env')

In [1]:
import requests
# Use the bing image search API to search for pictures of players
# Replace 'YOUR_API_KEY' with your actual API key
api_key = os.getenv('BING_API_KEY')

# Set the search query and other parameters
search_query = 'Victor Wembanyama headshot'
count = 5

# API endpoint URL
url = 'https://api.bing.microsoft.com/v7.0/images/search'

# Set the headers with the API key
headers = {'Ocp-Apim-Subscription-Key': api_key}

# Set the query parameters
params = {
    'q': search_query,
    'count': count
}

# Make the API call
response = requests.get(url, headers=headers, params=params)

# Get the JSON response
data = response.json()

# Extract the image URLs from the response
image_urls = [result['contentUrl'] for result in data['value']]

# Print the image URLs
for url in image_urls:
    print(url)


https://s.hdnux.com/photos/01/33/52/57/24030153/3/1200x0.jpg
https://www.proballers.com/media/cache/resize_300/ul/player/backup/sans-titre-1-1ed9f42b-f622-6b96-89ba-3faf8e459635.jpg
https://www.2kratings.com/wp-content/uploads/Victor-Wembanyama-2K-Rating.png
https://d1l5jyrrh5eluf.cloudfront.net/wp-content/uploads/2022/10/VictorW_Watermarked_02.jpg
https://eurospects.com/wp-content/uploads/2020/09/wembanyama.jpg


In [2]:
import pandas as pd
from io import StringIO

# Sample table data with Spurs players names in CSV format

table_str = """No.,Player,Pos,Ht,Wt,Birth Date,,Exp,College
33,Tre Jones,PG,6-1,185,January 8 2000,us,3,Duke
22,Malaki Branham,PG,6-4,180,May 12 2003,us,1,Ohio State
10,Jeremy Sochan,PF,6-8,230,May 20 2003,us,1,Baylor
30,Julian Champagnie,SF,6-8,220,June 29 2001,us,1,St. John's
16,Cedi Osman,SF,6-7,230,April 8 1995,mk,6,
1,Victor Wembanyama,C,7-4,209,January 4 2004,fr,R,
3,Keldon Johnson,SF,6-5,220,October 11 1999,us,4,Kentucky
23,Zach Collins,C,6-11,250,November 19 1997,us,5,Gonzaga
24,Devin Vassell,SG,6-5,200,August 23 2000,us,3,Florida State
14,Blake Wesley,SG,6-5,185,March 16 2003,us,1,Notre Dame
54,Sandro Mamukelashvili,C,6-11,240,May 23 1999,us,2,Seton Hall
26,Dominick Barlow,PF,6-9,221,May 26 2003,us,1,
4,Devonte' Graham,PG,6-1,195,February 22 1995,us,5,Kansas
28,Charles Bassey,C,6-10,235,October 28 2000,ng,2,Western Kentucky
25,Sidy Cissoko,SF,6-8,200,April 2 2004,fr,R,
7,David Duke Jr.  (TW),SG,6-5,205,October 13 1999,us,2,Providence
41,RaiQuan Gray  (TW),PF,6-8,260,July 7 1999,us,1,Florida State
15,Jamaree Bouyea  (TW),PG,6-2,180,June 27 1999,us,1,San Francisco"""

df = pd.read_csv(StringIO(table_str), sep=',', header=0)
df.head()


,No.,Player,Pos,Ht,Wt,Birth Date,Unnamed: 6,Exp,College
0,33,Tre Jones,PG,6-1,185,January 8 2000,us,3,Duke
1,22,Malaki Branham,PG,6-4,180,May 12 2003,us,1,Ohio State
2,10,Jeremy Sochan,PF,6-8,230,May 20 2003,us,1,Baylor
3,30,Julian Champagnie,SF,6-8,220,June 29 2001,us,1,St. John's
4,16,Cedi Osman,SF,6-7,230,April 8 1995,mk,6,NaN


In [8]:
import os
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, QualityForRecognition
import time


FACES_API_KEY = os.getenv('FACES_API_KEY')
FACES_ENDPOINT = os.getenv('FACES_ENDPOINT')
PERSON_GROUP_ID = "nbaplayers" # assign a random ID (or name it anything)

# Create an authenticated FaceClient.
face_client = FaceClient(FACES_ENDPOINT,
                        CognitiveServicesCredentials(FACES_API_KEY))

'''
Create the PersonGroup
'''
# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
person_groups = [group.name for group in face_client.person_group.list()]
if PERSON_GROUP_ID in person_groups:
    print(f"Person group {PERSON_GROUP_ID} already exists.")
else:
    print(f"Creating person group {PERSON_GROUP_ID}...")
    face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID, recognition_model='recognition_04')


# Retry parameters
max_retries = 3
retry_delay = 2  # seconds

existing_players = [player.name for player in face_client.person_group_person.list('nbaplayers')]

# Retry loop
for player in df['Player'].tolist():
    if player in existing_players:
        print(f"Skipping {player} as it already exists.")
        continue
    else:
        print(f"Creating person for {player}...")
        retries = 0
        while retries < max_retries:
            try:
                face_client.person_group_person.create(PERSON_GROUP_ID, name=player)
                break  # Exit the loop if create() succeeds
            except Exception as e:
                print(f"Error creating person: {e}")
                retries += 1
                if retries < max_retries:
                    print(f"Retrying after {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    print("Max retries reached. Skipping person creation.")


Person group: nbaplayers
Person group nbaplayers already exists.
Skipping Tre Jones as it already exists.
Skipping Malaki Branham as it already exists.
Skipping Jeremy Sochan as it already exists.
Skipping Julian Champagnie as it already exists.
Skipping Cedi Osman as it already exists.
Skipping Victor Wembanyama as it already exists.
Skipping Keldon Johnson as it already exists.
Skipping Zach Collins as it already exists.
Skipping Devin Vassell as it already exists.
Skipping Blake Wesley as it already exists.
Creating person for Sandro Mamukelashvili...
Creating person for Dominick Barlow...
Creating person for Devonte' Graham...
Creating person for Charles Bassey...
Creating person for Sidy Cissoko...
Creating person for David Duke Jr.  (TW)...
Creating person for RaiQuan Gray  (TW)...
Creating person for Jamaree Bouyea  (TW)...


In [9]:

def search_images(search_query, count):
    # Replace 'YOUR_API_KEY' with your actual API key
    api_key = os.getenv('BING_API_KEY')

    # API endpoint URL
    url = 'https://api.bing.microsoft.com/v7.0/images/search'

    # Set the headers with the API key
    headers = {'Ocp-Apim-Subscription-Key': api_key}

    # Set the query parameters
    params = {
        'q': search_query,
        'count': count
    }

    # Make the API call
    response = requests.get(url, headers=headers, params=params)

    # Get the JSON response
    data = response.json()

    # Extract the image URLs from the response
    image_urls = [result['contentUrl'] for result in data['value']]

    # Return the image URLs
    return image_urls


In [10]:
df['faces'] = df['Player'].apply(lambda x: search_images(f"{x} Spurs headshot", 10))

In [18]:
pers_dict = {}
for pers in face_client.person_group_person.list('nbaplayers'):
    pers_dict[pers.name]= pers.person_id

print(pers_dict)

{'Devin Vassell': '073485f2-01fd-4f82-8598-ac9de142f45f', 'David Duke Jr.  (TW)': '07f9191a-69e8-4765-8331-3266987e917b', 'Zach Collins': '1a6bcdaf-6727-4182-8527-2bc72642ef3c', 'Julian Champagnie': '5391e31e-dacb-4e65-8aa6-abca254d371f', 'Jeremy Sochan': '55241cf6-4ab0-4368-9bd0-8c73a0839b51', 'Sandro Mamukelashvili': '55822fb9-454d-4fb1-994b-d97e4a0cc6bc', 'RaiQuan Gray  (TW)': '6667bd63-ec71-4a7d-8f5b-9dfad909266a', 'Victor Wembanyama': '7ae200de-32e6-4f80-9ef3-6ec19fe1b1ce', 'Malaki Branham': '83dc1aa7-f462-4e25-b5d7-7dadb20d6031', 'Charles Bassey': '8b7422d7-d933-4bb0-9d56-94a64c3eddd9', 'Jamaree Bouyea  (TW)': 'b80001dd-4019-4c23-8c8b-450d9d1442d5', 'Keldon Johnson': 'c113a765-a62d-4f79-9df6-7d04ff23dcac', 'Tre Jones': 'c6ec5ac4-cc0c-4986-ab80-37836007c456', 'Dominick Barlow': 'c92783d2-ae7e-4947-a972-be2e245b092a', 'Cedi Osman': 'c969c722-cf66-44e5-9c1a-2434dc3c3453', 'Blake Wesley': 'e33fe7e8-f038-4cf0-af1e-18acaa663d4e', "Devonte' Graham": 'ed385e26-214f-49af-a05d-731d1a339d4d

In [19]:
df['person_id'] = df['Player'].apply(lambda x: pers_dict[x] if x in pers_dict else None)

In [28]:
def add_faces_to_person(face_client, person_group_id, person_id, image_urls):
    for image_url in image_urls:
        sufficientQuality = True
        detected_faces = face_client.face.detect_with_url(url=image_url, detection_model='detection_03', recognition_model='recognition_04', return_face_attributes=['qualityForRecognition'])
        for face in detected_faces:
            if face.face_attributes.quality_for_recognition != QualityForRecognition.high:
                sufficientQuality = False
                print("{} has insufficient quality".format(face))
                break
            try:
                face_client.person_group_person.add_face_from_url(person_group_id, person_id, image_url)
                print("face {} added to person {}".format(face.face_id, person_id))
            except:
                continue
        if not sufficientQuality: continue


In [29]:
df.apply(lambda row: add_faces_to_person(face_client, PERSON_GROUP_ID, row['person_id'], row['faces']), axis=1)


face 6837896c-261d-4163-aee4-6cfb0a327f07 added to person c6ec5ac4-cc0c-4986-ab80-37836007c456
face 4bdc1887-cef5-4322-80de-44613078a30c added to person c6ec5ac4-cc0c-4986-ab80-37836007c456
face 514d604c-a88f-4b2c-85ec-8ec90c7157b4 added to person c6ec5ac4-cc0c-4986-ab80-37836007c456
face 2be11b1c-0791-40b6-bd0d-87af76230a21 added to person c6ec5ac4-cc0c-4986-ab80-37836007c456
face 37720a70-e77c-4873-9fea-64f5716b0517 added to person c6ec5ac4-cc0c-4986-ab80-37836007c456
{'additional_properties': {}, 'face_id': 'bab073e9-96e2-4e0c-ac7f-6662f2283926', 'recognition_model': None, 'face_rectangle': <azure.cognitiveservices.vision.face.models._models_py3.FaceRectangle object at 0x7f2b1b3f5490>, 'face_landmarks': None, 'face_attributes': <azure.cognitiveservices.vision.face.models._models_py3.FaceAttributes object at 0x7f2b1b3f4790>} has insufficient quality
{'additional_properties': {}, 'face_id': '2a43c21c-51ea-4c99-9937-2d46475b3edc', 'recognition_model': None, 'face_rectangle': <azure.cog

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
dtype: object

In [30]:
'''
Train PersonGroup
'''
# Train the person group
print("pg resource is {}".format(PERSON_GROUP_ID))
rawresponse = face_client.person_group.train(PERSON_GROUP_ID, raw= True)
print(rawresponse)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
        sys.exit('Training the person group has failed.')
    time.sleep(5)

pg resource is nbaplayers
Training status: TrainingStatusType.running.

Training status: TrainingStatusType.succeeded.



In [43]:
# Detect faces
face_ids = []

local_file = 'extracted_images/image_cropped_3.png'

with open(local_file, "rb") as f:
    faces_local = face_client.face.detect_with_stream(f, detection_model='detection_03', recognition_model='recognition_04', return_face_attributes=['qualityForRecognition'])

for face in faces_local:
    # Only take the face if it is of sufficient quality.
    if face.face_attributes.quality_for_recognition == QualityForRecognition.high or face.face_attributes.quality_for_recognition == QualityForRecognition.medium:
        face_ids.append(face.face_id)

# Identify faces
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
print('Identifying faces in image')
if not results:
    print('No person identified in the person group')
for identifiedFace in results:
    if len(identifiedFace.candidates) > 0:
        print('Person is identified for face ID {} in image, with a confidence of {}.'.format(identifiedFace.face_id, identifiedFace.candidates[0].confidence)) # Get topmost confidence score

        # Verify faces
        verify_result = face_client.face.verify_face_to_person(identifiedFace.face_id, identifiedFace.candidates[0].person_id, PERSON_GROUP_ID)
        print('verification result: {}. confidence: {}'.format(verify_result.is_identical, verify_result.confidence))
        player = df.loc[df['person_id'] == identifiedFace.candidates[0].person_id, 'Player'].values[0]
        print(f"The identified player is: {player}")
    else:
        print('No person identified for face ID {} in image.'.format(identifiedFace.face_id))

Identifying faces in image
Person is identified for face ID fbd00eba-d166-46ca-a4e8-93c0daaeaa06 in image, with a confidence of 0.93724.
verification result: True. confidence: 0.93724
